In [6]:
from datasets import load_dataset
import pandas as pd
from itertools import islice

In [7]:
book_iter = load_dataset("bookcorpus", "plain_text", split='train', streaming=True, trust_remote_code=True)
book_sample = list(islice(book_iter, 25000))

In [8]:
sports_iter = load_dataset("Chrisneverdie/OnlySports_Dataset", split='train', streaming=True)
sports_sample = list(islice(sports_iter, 25000))

Resolving data files:   0%|          | 0/678 [00:00<?, ?it/s]

In [32]:
movie_df = pd.read_csv("datasets-to-merge/IMDB Dataset.csv")
movie_sample = movie_df.sample(n=25000, random_state=42).reset_index(drop=True)

In [10]:
book_df = pd.DataFrame(book_sample)
sports_df = pd.DataFrame(sports_sample)

In [33]:
movie_sample = movie_sample.rename(columns={'review': 'text'})
movie_sample['topic'] = 'movie'
movie_sample = movie_sample.drop(columns=['sentiment'])
print(movie_sample.head())

                                                text  topic
0  I really liked this Summerslam due to the look...  movie
1  Not many television shows appeal to quite as m...  movie
2  The film quickly gets to a major chase scene w...  movie
3  Jane Austen would definitely approve of this o...  movie
4  Expectations were somewhat high for me when I ...  movie


In [28]:
book_df['topic'] = 'book'
print(book_df.head())

                                                text topic
0  usually , he would be tearing around the livin...  book
1  but just one look at a minion sent him practic...  book
2  that had been megan 's plan when she got him d...  book
3  he 'd seen the movie almost by mistake , consi...  book
4  she liked to think being surrounded by adults ...  book


In [ ]:
sports_df = sports_df.drop(columns=['url', 'token_count'])
sports_df['topic'] = 'sports'
print(sports_df.head())


                                                text   topic
0  San Francisco 49ers cornerback Shawntae Spence...  sports
1  White House Has a Plan to Tackle 3 Separate Sc...  sports
2  6:29 p.m. ET: ABC’s David Wright tweeted:\n3:4...  sports
3  There’s no doubt of my affections for the Rich...  sports
4  This is one of the few weekends every year tha...  sports


In [45]:
print(len(movie_sample), len(book_df), len(sports_df))

25000 25000 25000


In [46]:
combined_df = pd.concat([book_df, sports_df, movie_sample], ignore_index=True)

In [47]:
combined_df.to_csv("topic-analysis-dataset.csv", index=False)

In [51]:
df = pd.read_csv("topic-analysis-dataset.csv")
unique_topics = df['topic'].unique()
print(f"Unique topics: {unique_topics}")
print(f"Number of unique topics: {len(unique_topics)}")
topic_counts = df['topic'].value_counts()
print(topic_counts)

Unique topics: ['book' 'sports' 'movie']
Number of unique topics: 3
topic
book      25000
sports    25000
movie     25000
Name: count, dtype: int64
